## Applying Lasso to perform feature selection

In [2]:
import numpy as np
import scipy.stats
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

Importing Dataset

### Boston dataset

In [3]:
from sklearn.datasets import load_boston
X,y = load_boston(return_X_y=True)
features = load_boston()['feature_names']

c:\Users\91779\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the

### Wine dataset

In [4]:
from sklearn.datasets import load_wine
X,y = load_wine(return_X_y=True)
features = load_wine()['feature_names']

### Iris dataset

In [5]:
from sklearn.datasets import load_iris
X,y = load_iris(return_X_y=True)
features = load_iris()['feature_names']

### Digits dataset

In [6]:
from sklearn.datasets import load_digits
X,y = load_digits(return_X_y=True)
features = load_digits()['feature_names']

Data split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Pipleline for scaling data

In [8]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

we are going to test several values from 0.1 to 10 with 0.1 step. For each value, we calculate the average value of the mean squared error in a 5-folds cross-validation and select the value of α that minimizes such average performance metrics.

In [9]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

We use neg_mean_squared_error because the grid search tries to maximize the performance metrics, so we add a minus sign to minimize the mean squared error.

In [10]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-3.629 total time=   0.0s
[CV 2/5] END .................model__alpha=0.1;, score=-3.915 total time=   0.0s
[CV 3/5] END .................model__alpha=0.1;, score=-4.127 total time=   0.0s
[CV 4/5] END .................model__alpha=0.1;, score=-4.161 total time=   0.0s
[CV 5/5] END .................model__alpha=0.1;, score=-4.705 total time=   0.0s
[CV 1/5] END .................model__alpha=0.2;, score=-4.154 total time=   0.0s
[CV 2/5] END .................model__alpha=0.2;, score=-4.421 total time=   0.0s
[CV 3/5] END .................model__alpha=0.2;, score=-4.885 total time=   0.0s
[CV 4/5] END .................model__alpha=0.2;, score=-4.817 total time=   0.0s
[CV 5/5] END .................model__alpha=0.2;, score=-5.045 total time=   0.0s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-4.772 total time=   0.0s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

The best value for α can be found using,

In [11]:
best_alhpa = search.best_params_
print("Best value for alpha is = ", '%.6f'%float(best_alhpa['model__alpha']))

Best value for alpha is =  0.100000


Now, we have to get the values of the coefficients of Lasso regression. The importance of a feature is the absolute value of its coefficient,

In [12]:
coefficients = search.best_estimator_.named_steps['model'].coef_
importance = np.abs(coefficients)
print(importance)

[0.         0.         0.         0.05752486 0.02467769 0.
 0.         0.02555828 0.         0.         0.16912822 0.
 0.26492136 0.         0.14839312 0.         0.         0.
 0.41873888 0.         0.39662162 0.         0.         0.
 0.         0.28950904 0.         0.45674386 0.35150967 0.409443
 0.         0.         0.         0.25354802 0.         0.58469506
 0.         0.17711474 0.         0.         0.         0.0044046
 0.         0.         0.33296506 0.20888257 0.         0.
 0.         0.         0.         0.22205225 0.96016851 0.19538036
 0.         0.         0.         0.         0.         0.
 0.10861776 0.01063974 0.         0.14104189]


We set a threshold value to allow us to control the level of impactfulness required from nthe features

In [13]:
threshold = 0

According to the set threshold,

The features that survived the Lasso regression are,

In [14]:
np.array(features)[importance > threshold]

array(['pixel_0_3', 'pixel_0_4', 'pixel_0_7', 'pixel_1_2', 'pixel_1_4',
       'pixel_1_6', 'pixel_2_2', 'pixel_2_4', 'pixel_3_1', 'pixel_3_3',
       'pixel_3_4', 'pixel_3_5', 'pixel_4_1', 'pixel_4_3', 'pixel_4_5',
       'pixel_5_1', 'pixel_5_4', 'pixel_5_5', 'pixel_6_3', 'pixel_6_4',
       'pixel_6_5', 'pixel_7_4', 'pixel_7_5', 'pixel_7_7'], dtype='<U9')

The features that did not survive the Lasso regression are,

In [15]:
np.array(features)[importance <= threshold]

array(['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_5', 'pixel_0_6',
       'pixel_1_0', 'pixel_1_1', 'pixel_1_3', 'pixel_1_5', 'pixel_1_7',
       'pixel_2_0', 'pixel_2_1', 'pixel_2_3', 'pixel_2_5', 'pixel_2_6',
       'pixel_2_7', 'pixel_3_0', 'pixel_3_2', 'pixel_3_6', 'pixel_3_7',
       'pixel_4_0', 'pixel_4_2', 'pixel_4_4', 'pixel_4_6', 'pixel_4_7',
       'pixel_5_0', 'pixel_5_2', 'pixel_5_3', 'pixel_5_6', 'pixel_5_7',
       'pixel_6_0', 'pixel_6_1', 'pixel_6_2', 'pixel_6_6', 'pixel_6_7',
       'pixel_7_0', 'pixel_7_1', 'pixel_7_2', 'pixel_7_3', 'pixel_7_6'],
      dtype='<U9')

## Applying correlation to do the same

In [16]:
r_col = y_train
n_ft = len(X_train[0])
n_cof = [None]*n_ft

for i in range(n_ft):
    i_col = df = X_train[:,i]
    n_cof[i] = scipy.stats.pearsonr(r_col, i_col)[0]

importance_corr = np.abs(np.round(n_cof,4))

print(importance_corr)

[      nan 6.570e-02 1.340e-02 3.380e-02 7.950e-02 1.835e-01 2.178e-01
 1.182e-01 1.700e-03 3.130e-02 9.490e-02 1.034e-01 2.618e-01 4.660e-02
 2.326e-01 9.120e-02 2.700e-02 4.860e-02 7.910e-02 1.086e-01 1.575e-01
 9.280e-02 9.910e-02 3.840e-02 2.730e-02 1.601e-01 6.890e-02 2.758e-01
 2.199e-01 1.863e-01 3.190e-02 6.000e-03       nan 2.057e-01 1.600e-02
 2.847e-01 1.826e-01 1.435e-01 3.690e-02       nan 1.040e-02 1.143e-01
 1.020e-01 6.500e-03 2.000e-04 1.950e-02 5.550e-02 4.980e-02 1.290e-02
 2.600e-02 1.472e-01 1.832e-01 3.938e-01 2.070e-01 2.550e-02 7.480e-02
 2.430e-02 5.940e-02 1.600e-03 3.640e-02 2.140e-01 7.660e-02 1.011e-01
 1.720e-01]


c:\Users\91779\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


selecting some small number to check if the most significant are always being selected

In [19]:
n_imp = 5

Finding the highest impactful features wrt correlation coefficient values

In [20]:
imp_arr = importance_corr*[importance > threshold]
rel_imp = imp_arr[np.nonzero(imp_arr)]

print(np.sort(importance_corr)[::-1][:n_imp])

print(np.sort(rel_imp)[::-1][:n_imp])

[   nan    nan    nan 0.3938 0.2847]
[   nan    nan    nan 0.3938 0.2847]
